In [2]:
from network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_direct_downmapping_with_conv import SS_Direct_Downmapping_Conv_Model
import importlib

import numpy as np

from network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_direct_downmapping import SS_Direct_Downmapping_Model
from network_models.soundsream_models_and_utils.clip_like.encoder.ss_encoder_downmapping import EncoderDownmapping
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 8
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/downmapping/0_0218/"
epochs =1500
save_every = 200
start_lr = 6e-6
gc.collect()
seed = 300

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings_version0_12_1.pkl", device=device)

#model = SS_Direct_Downmapping_Model(dropout=0.2, output=1024, start_dim=512 * 175).to(device)
#model = SS_Direct_Downmapping_Conv_Model(dropout=0.2, output=1024, x_size=512, y_size=175).to(device)
torch.manual_seed(seed)
model = EncoderDownmapping(embed_dim=512, n_heads=4, ff_dim=2, n_layers=1, dropout=0.2, output=1024, max_seq_len=175).to(device)

In [4]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundsream_models_and_utils.clip_like.mapping_down.ss_driect_dm_trainer as sset
importlib.reload(sset)

epochs =2001
#start_lr = 4e-6
enc_trainer = sset.SSDirectDMTrainer(is_encoder=True, batch_size=batch_size, num_epochs=epochs, model_path=models_dir, save_model_every=save_every, lr=start_lr, dataset=data_set, device=device, model=model)
gc.collect()
enc_trainer.train() # 200 +500 + 500 +500+500

Epoch 1
-------------------------------
loss: 1.881667  [    0/ 6092]
loss: 1.932698  [ 1400/ 6092]
loss: 1.960277  [ 2800/ 6092]
loss: 1.962337  [ 4200/ 6092]
loss: 2.026007  [ 5600/ 6092]
916.7197245359421
Epoch 2
-------------------------------
loss: 2.030303  [    0/ 6092]
loss: 1.953770  [ 1400/ 6092]
loss: 1.973231  [ 2800/ 6092]
loss: 1.976374  [ 4200/ 6092]
loss: 1.976968  [ 5600/ 6092]
914.2783634662628
Epoch 3
-------------------------------
loss: 1.952375  [    0/ 6092]
loss: 1.985815  [ 1400/ 6092]
loss: 1.944740  [ 2800/ 6092]
loss: 2.015264  [ 4200/ 6092]
loss: 1.929853  [ 5600/ 6092]
912.2420064210892
Epoch 4
-------------------------------
loss: 1.841490  [    0/ 6092]
loss: 1.953311  [ 1400/ 6092]
loss: 1.990728  [ 2800/ 6092]
loss: 1.961176  [ 4200/ 6092]
loss: 1.974828  [ 5600/ 6092]
911.7231410741806
Epoch 5
-------------------------------
loss: 1.992179  [    0/ 6092]
loss: 1.942945  [ 1400/ 6092]
loss: 1.909635  [ 2800/ 6092]
loss: 1.925145  [ 4200/ 6092]
loss: 1.

In [5]:
# import torch
# import gc
# torch.cuda.empty_cache()
# gc.collect()
#
# import numpy as np
# from audiolm_pytorch import SoundStream, SoundStreamTrainer
#
# soundstream = SoundStream(
#
#     codebook_size = 2048,
#     rq_num_quantizers = 12,
#     attn_window_size = 128,       # local attention receptive field at bottleneck
#     attn_depth = 3
#     #target_sample_hz=16000
# )
#
# #soundstream.load("../content/soundstream/verision0.12.1/00_10000_1e-4_bs6_gae8_dml320-32/soundstream.42500.pt")
#
# trainer = SoundStreamTrainer(
#     soundstream,
#     folder =  '/home/ckwdani/Music/libri',
#     #'/home/ckwdani/Music/train-clean-100',
#     batch_size = 6,
#     grad_accum_every = 8,         # effective batch size of 32
#     data_max_length = 320 * 32,
#     save_results_every=1000,
#     num_train_steps = 60001,
#     save_model_every= 500,
#     results_folder = '../content/soundstream/verision0.12.1/RQ12_CBS_2048_attnD_3/00_10000_1e-4_bs6_gae8_dml320-32',
#     lr=1e-4
# #    lr = 5e-5,
#     #lr = 5e-5
#     #standard lr 3e-4,
# ).cuda()
#
# trainer.train()

training with dataset of 267178 samples and validating with randomly splitted 14063 samples


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 9.77 GiB total capacity; 6.43 GiB already allocated; 64.31 MiB free; 6.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [33]:
# from network_models.soundsream_models_and_utils.ss___util_class_batches_sampler import ClassBatchesSampler
# np1 = data_set.encoded_dataset.encodedData[data_set.encoded_dataset.labelcolumn].to_numpy()
# from torch.utils.data import DataLoader
# #
# # l = [np1[i][0] for i in range(len(np1))]
# #
# #
# dl = DataLoader(data_set, batch_sampler=ClassBatchesSampler(np1, num_class_samples=2, shuffle=False))
# #
# #
# for batch, (X, z) in enumerate(dl):
#
#     print(torch.tensor_split(torch.argmax(z, dim=1), 2))
#     z1, z2 = torch.tensor_split(z, 2)
#     pred = model(X.cuda())
#     pred1, pred2 = torch.tensor_split(pred, 2)
#     dp = torch.matmul(pred1, pred2.t())
#     #dp = pred1 @ pred2.t()
#
#
#     print(torch.dot(pred1[0],pred2[0]))
#     # print(torch.dot(pred[1],pred2[0]))
#     # print(torch.dot(pred[2],pred2[0]))
#     # print(torch.dot(pred[3],pred2[0]))
#     # print(torch.dot(pred[4],pred2[0]))
#     # print(torch.dot(pred[5],pred2[0]))
#     # print(torch.dot(pred[7],pred[0]))
#     #dp = torch.matmul(z1, z2.T) / (z1.norm(dim=-1, keepdim=True) * z2.norm(dim=-1, keepdim=True))
#     #print(dp)
#     print(dp)
#     print(torch.sum(dp[0, :]))
#
#     #print(X[0][0][0][170])
#
#     if batch == 0:
#         break
#
# (len(dl)/(7*2))-2 == 462

(tensor([0, 1, 2, 3, 4, 5, 6]), tensor([0, 1, 2, 3, 4, 5, 6]))
tensor(0.0257, device='cuda:0', grad_fn=<DotBackward0>)
tensor([[ 0.0257,  0.0682,  0.1137,  0.1003,  0.0861,  0.0434,  0.1270],
        [ 0.0364,  0.1691,  0.0499,  0.0520,  0.0445,  0.0215,  0.0879],
        [ 0.0471,  0.2021,  0.1437,  0.1197,  0.0319,  0.0232,  0.0524],
        [-0.0349,  0.1412, -0.0580,  0.1619,  0.0691,  0.0561,  0.1994],
        [ 0.1065,  0.1545,  0.1924,  0.1211,  0.1037,  0.0426,  0.1208],
        [ 0.1051,  0.1988,  0.1682,  0.2198,  0.1852,  0.0045,  0.1342],
        [ 0.0185,  0.0735,  0.0420,  0.0426,  0.0786,  0.0026,  0.5665]],
       device='cuda:0', grad_fn=<MmBackward0>)
tensor(0.5645, device='cuda:0', grad_fn=<SumBackward0>)


True